In [6]:
#!pip install psycopg2

  Using cached psycopg2-2.9.3.tar.gz (380 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-kauzn1_a/psycopg2.egg-info
      writing /tmp/pip-pip-egg-info-kauzn1_a/psycopg2.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-kauzn1_a/psycopg2.egg-info/dependency_links.txt
      writing top-level names to /tmp/pip-pip-egg-info-kauzn1_a/psycopg2.egg-info/top_level.txt
      writing manifest file '/tmp/pip-pip-egg-info-kauzn1_a/psycopg2.egg-info/SOURCES.txt'
      
      Error: pg_config executable not found.
      
      pg_config is required to build psycopg2 from source.  Please add the directory
      containing pg_config to the $PATH or specify the full executable path with the
      option:
      
          python setup.py build_ext --pg-config /path/to/pg_config build

In [2]:
import datetime as dt
import sys
import os
import boto3
import pandas as pd
from boto3.session import Session
from io import StringIO
import numpy as np

# declaring some constants
bucket = 'calval-reference-data'
session = Session()
client = session.client('s3')

# gsheet
import pickle
import os.path
# from googleapiclient.discovery import build
# from google_auth_oauthlib.flow import InstalledAppFlow
# from google.auth.transport.requests import Request

import json
#import psycopg2 as pg
import pandas.io.sql as psql
import warnings
warnings.filterwarnings("ignore")

In [3]:
def listfiles(bucket, prefix=''):
    session = Session()
    client = session.client('s3')
    """List files in specific S3 URL"""
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Prefix=prefix, Delimiter='/'):
        for content in result.get('Contents', []):
            yield content.get('Key')

            
def get_gsheet(sheet_id, sheet_name, sheet_range):
    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    sheet_range = sheet_name + sheet_range
    creds = None
    with open('/home/ec2-user/SageMaker/adse/Met_Facts/Radiometry_Mark2/swdw cosine/token_3.pickle', 'rb') as token:
        creds = pickle.load(token)
    if not creds:
        print('token file not found')
    service = build('sheets', 'v4', credentials=creds)
    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=sheet_id,
                                range=sheet_range).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        df = pd.DataFrame(values)
        df = df.rename(columns=df.iloc[0])
        df = df.drop([0])
        df.reset_index(inplace=True, drop=True)
    return df

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
start_date = '2020-01-01'

def get_mark_calval_deployments(which: str = "all"):
    """Fetch pandas df of mark calval deployment data from calval correspondence table
    Args:
        which - determine which deployment data to fetch, 'current', 'historical' or 'all'
    returns:
        deployment_data - pandas df with deployment data
    """

    url_base = "https://docs.google.com/spreadsheets/d/e"
    doc_id = "2PACX-1vTwPZ7gv05wgGVvaoL0H_IfsLmzXx_nLwUpmWSHwn1Rl5hD0_L8Ur-iuMGueavQitRMMteqLtoFHpqV"
    current_url = f"{url_base}/{doc_id}/pub?gid=0&single=true&output=csv"
    historical_url = f"{url_base}/{doc_id}/pub?gid=1303173661&single=true&output=csv"
    if which == "current":
        urls = [current_url]
    elif which == "historical":
        urls = [historical_url]
    elif which == "all":
        urls = [current_url, historical_url]
    else:
        raise Exception(
            f"Unidentified Data Source: '{which}'. 'which' argument must be set to 'current', 'historical' or 'all' "
        )

    # fetch data
    deployment_data = pd.concat([pd.read_csv(url, parse_dates=["start_date", "end_date"]) for url in urls])
    # format end dates for active devices
    deployment_data["end_date"] = deployment_data["end_date"].apply(
        lambda x: dt.datetime.utcnow() if x == "still active" else x
    )
    deployment_data["end_date"] = pd.to_datetime(deployment_data["end_date"], errors='coerce')
    # return data of interest
    return deployment_data[["site", "mark2", "start_date", "end_date"]]

In [5]:
sites_mark =get_mark_calval_deployments()
sites_mark

,site,mark2,start_date,end_date
0,AOSSB,C005252,2020-12-16 16:33:04,2022-07-29 21:12:00.354660
1,Ardoyne,C007852,2021-07-01 15:08:55,2022-07-29 21:12:00.354662
2,Ardoyne,C007880,2021-07-01 14:16:21,2022-07-29 21:12:00.354663
3,AU-Cpr,C003340,2020-06-10 03:53:01,2022-07-29 21:12:00.354663
4,AU-Rob,C006271,2021-11-17 18:45:39,2022-07-29 21:12:00.354663
...,...,...,...,...
130,AU-Rob,C003445,2020-06-03 12:14:44,2021-11-18 02:03:39.000000
131,JF_San_Jose,C006108,2021-01-13 23:10:06,2022-07-15 20:33:00.000000
132,HQF,C004251,2020-12-26 21:44:35,2022-07-15 17:00:00.000000
133,HQF,C004919,2020-12-31 08:31:19,2022-07-15 17:00:00.000000


### Get Info On Ref and Mark

#### Data in S3

In [6]:
file_save_path = 's3://arable-adse-dev/ald/Desktop/Met_Facts/lfw/methods_comparison_feb2022/'

### device ID for mark devices

In [7]:


devices = ['C004146',
'C007880',
'C004301',
'C006056',
'C002526',
'C004838',
'C002681',
'C002617',
'C005217',
'C005337',
'C002759',
'C002496',
'C003324',
'C003088',
'C006826',
'C002472',
'C003017',
'C004854',
'C006740',
'C004313']

site_device =['Ardoyne_C004146',
'Ardoyne_C007880',
'Ardoyne_C004301',
'Cabure_C006056',
'Field&Fork_C004838',
'Field&Fork_C002681',
'Field&Fork_C002617',
'Field&Fork_C005217',
'Salinas_C005337',
'Salinas_C002759',
'Santa_Rosa_JimE_C002496',
'Santa_Rosa_JimE_C003324',
'Cabure_C003088',
'US-Los_C006826',
'US-Los_C003017',
'US-Los_C004854',
'Yara_C006740'
]



#### Read Mark data from S3

In [8]:
# for device in device:
#     try:
#         mark = pd.read_csv(f"s3://arable-adse-dev/ald/Desktop/Met_Facts/lfw/methods_comparison_feb2022/mark/joined/{device}.csv")
        
#     except:
#         mark_5min = pd.read_csv(f'{file_save_path}/mark/5min/{device}.csv')
#         mark_5min['time'] = mark_5min['time'].astype('datetime64[ns]')
#         mark_5min['hour'] = mark_5min['time'].dt.floor('h')
#         mark_hourly = pd.read_csv(f'{file_save_path}/mark/hourly/{device}.csv')
#         mark_hourly['time'] = mark_hourly['time'].astype('datetime64[ns]')
#         mark_hourly['hour'] = mark_hourly['time'].dt.floor('h')
#         mark_hourly = mark_hourly[['hour', 'precip', 'rh_hourly']]
#         mark = mark_5min.merge(mark_hourly, on='hour', how='left')

In [9]:
# combined_df = []
# for device in devices:
#     df = pd.read_csv(f"s3://arable-adse-dev/ald/Desktop/Met_Facts/lfw/methods_comparison_feb2022/mark/joined/{device}.csv")
#     #df = df.sort_values(by='time')
#     combined_df.append(df)
# combined_df = pd.concat(combined_df)
# combined_df.to_csv(f"s3://arable-adse-dev/ald/Desktop/Met_Facts/lfw/methods_comparison_feb2022/mark/combined_data.csv", index=False)

# combined_df = pd.read_csv(f"s3://arable-adse-dev/ald/Desktop/Met_Facts/lfw/methods_comparison_feb2022/mark/combined_data.csv")

# combined_df['device'].nunique()

# combined_df.head()

#### Get refernce data info 

In [10]:
ref_path = 's3://calval-reference-data/csv_p2'

In [11]:
calval_spec_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vT2FauubHrvZVInURBAYqNjuoGxaS1Fcp4iwvYM4AdBlkvxof4rHWoYNBZ9yT6LePWyCD1Fp04-KQU4/pub?gid=59898478&single=true&output=csv"
all_deployment_info = pd.read_csv(calval_spec_url)
sites = all_deployment_info[all_deployment_info['new_column']=="ref_lfw"]
# leaf_wetness_sites = leaf_wetness_site_info['site'].unique()
# print(leaf_wetness_sites)



In [12]:
sites

,site,orig_column,orig_units,new_column,new_units,use,measurement,time_res,time_edge,source,sensor,notes,metadata_version,training_data_suitable
363,Ardoyne,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,5.0,right,zentra,PHYTOS 31,NaN,1.0,NaN
394,Santa_Rosa_JimE,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,5.0,right,zentra,PHYTOS 31,Mark Level,1.0,NaN
501,Field&Fork,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,5.0,right,zentra,PHYTOS 31,at height of 115 cm,1.0,NaN
518,Salinas,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,5.0,right,zentra,PHYTOS 31,Mark Level,1.0,NaN
534,UCBG,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,5.0,right,zentra,PHYTOS 31,Mark Level,1.0,NaN
666,US-Los,leaf_wetness,mV,ref_lfw,mV,yes,Leaf wetness,1.0,right,us_los_sensor_suite,Phytos,US-Los_lfw_1,1.0,NaN
682,Santa_Rosa_JimE,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,10.0,TBD,pessl_imetos_3_3,Pessl iMetos 3.3,NaN,1.0,NaN
727,Cabure,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,5.0,right,zentra,PHYTOS 31,@ 150cm,1.0,NaN
961,Yara,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,5.0,right,growers_standard,PHYTOS 31,@ 50cm,1.0,NaN
1061,UCD_Almond,Leaf Wetness,min,ref_lfw,h,yes,Leaf Wetness Duration,5.0,right,ET100,PHYTOS 31,Sensor height is 450cm,1.0,NaN


In [13]:
ref_info = pd.DataFrame()
for i, row in sites.iterrows():
    site = row['site']
    if site == 'Cabure':
        prefix = 'ald/air_temp_sites/cal_sites_2019/Cabure/csv_p2/leaf wetness sensors/'
        tfilelist = listfiles('arable-adse-dev', prefix=prefix)
    # these sites are moved to the new directory already
    else:
        source = row['source']
        prefix = f'csv_p2/{site}/{source}/'
        tfilelist = listfiles('calval-reference-data', prefix=prefix)
    filelist = []
    start_dates = []
    end_dates = []
    min_start_date = dt.datetime.strptime('30000101', '%Y%m%d')
    max_end_date = dt.datetime.strptime('10000101', '%Y%m%d')
    initial = 1
    for f in tfilelist:
        # f looks like "s3://calval-reference-data/csv_p2/Cabure/zentra/Cabure__p2_5min_202104272005_202108062355.csv"
        if f.endswith(".csv"):
            t = str(f).split("/")
            t = t[-1]
            if site == 'Cabure':
                cur_start_date = dt.datetime.strptime(t.split('_')[-3], '%Y%m%d')
                cur_end_date = dt.datetime.strptime(t.split('_')[-2].split('.')[0], '%Y%m%d')                
            else:
                cur_start_date = dt.datetime.strptime(t.split('_')[-2], '%Y%m%d%H%M')
                cur_end_date = dt.datetime.strptime(t.split('_')[-1].split('.')[0], '%Y%m%d%H%M')
            if ((initial==1) | (cur_start_date < min_start_date)):
                min_start_date = cur_start_date
            if ((initial==1) | (cur_end_date > max_end_date)):
                max_end_date = cur_end_date
            initial = 0
    ref_info = ref_info.append({'site':site, 'ref_start':min_start_date, 'ref_end':max_end_date}, ignore_index=True)
# add in North Canyon
ref_info = ref_info.append({'site':'North_Canyon', 'ref_start':'2021-02-26 08:00:00', 'ref_end':'2021-04-29 06:45:00'}, ignore_index=True)
ref_info[['site', 'ref_start', 'ref_end']]   
    

,site,ref_start,ref_end
0,Ardoyne,2019-11-19 15:50:00,2022-07-24 23:55:00
1,Santa_Rosa_JimE,2019-09-05 20:30:00,2022-07-24 23:55:00
2,Field&Fork,2019-12-16 14:45:00,2022-07-24 23:55:00
3,Salinas,2019-12-05 00:30:00,2022-07-24 23:55:00
4,UCBG,3000-01-01 00:00:00,1000-01-01 00:00:00
5,US-Los,2020-01-06 17:09:00,2022-07-24 00:00:00
6,Santa_Rosa_JimE,2021-01-08 00:00:00,2022-07-08 23:50:00
7,Cabure,2021-04-28 00:00:00,2021-07-03 00:00:00
8,Yara,2021-05-13 18:15:00,2022-07-24 23:55:00
9,UCD_Almond,2021-04-22 00:00:00,2022-07-24 23:55:00


In [14]:
prefix

'csv_p2/rainpad_portland/zentra/'

##### Get Mark Info

In [15]:
sites_mark.head()

,site,mark2,start_date,end_date
0,AOSSB,C005252,2020-12-16 16:33:04,2022-07-29 21:12:00.354660
1,Ardoyne,C007852,2021-07-01 15:08:55,2022-07-29 21:12:00.354662
2,Ardoyne,C007880,2021-07-01 14:16:21,2022-07-29 21:12:00.354663
3,AU-Cpr,C003340,2020-06-10 03:53:01,2022-07-29 21:12:00.354663
4,AU-Rob,C006271,2021-11-17 18:45:39,2022-07-29 21:12:00.354663


In [16]:
sites_mark = sites_mark.rename(columns={'start_date':'mark_start', 'end_date':'mark_end'})

mark_info = sites_mark[sites_mark['site'].isin(ref_info.site.values.tolist())].sort_values(by=['site']).reset_index(drop=True)

mark_info[['mark_start', 'mark_end']] = mark_info[['mark_start','mark_end']].astype('datetime64[ns]')

#mark_info['mark_end'] = mark_info['mark_end'].astype('datetime64[ns]')


In [17]:
mark_info.dtypes

site                  object
mark2                 object
mark_start    datetime64[ns]
mark_end      datetime64[ns]
dtype: object

In [18]:
# for sites with multiple devices, limit to ones that are closest to lfw sensor
    # need to check: Field&Fork?
mark_info = mark_info.drop(mark_info[(mark_info['site'] == 'Santa_Rosa_JimE') & (~mark_info['mark2'].isin(['C002496', 'C003324']))].index)
mark_info = mark_info.drop(mark_info[(mark_info['site'] == 'Ardoyne') 
                                    & (~mark_info['mark2'].isin(['C004146', 'C004301', 'C007880']))].index)
mark_info = mark_info.drop(mark_info[(mark_info['site'] == 'Field&Fork') & (mark_info['mark2'].isin(['C004838']))].index)
# at Yara, devices C006717, C006759, C006759 that were active from 2021-03-15 20:39:25 2021-05-13 12:58:53 
# were there before lfw sensor were installed, we're only keeping this one device below that is documented 
# on CALVAL correspondence to be closest to the lfw sensor
mark_info = mark_info.drop(mark_info[(mark_info['site'] == 'Yara') 
                                     & (~mark_info['mark2'].isin(['C006740']))].index)
mark_info = mark_info.append({'site':'North_Canyon', 'mark2':'C004313', 'mark_start':'2021-02-26 08:00:00', 
                              'mark_end':'2021-04-29 06:45:00'}, ignore_index=True)
mark_info.reset_index(drop=True, inplace=True)
mark_info

,site,mark2,mark_start,mark_end
0,Ardoyne,C007880,2021-07-01 14:16:21,2022-07-29 21:12:00.354663
1,Ardoyne,C004146,2020-04-27 20:45:53,2020-08-27 20:10:42
2,Ardoyne,C004301,2020-08-12 19:38:53,2021-07-01 00:00:00
3,Cabure,C006056,2021-04-28 20:43:01,2022-07-29 21:12:00.354664
4,Field&Fork,C002617,2019-09-11 19:55:10,2020-04-01 01:34:53
5,Field&Fork,C002526,2019-07-09 21:54:24,2019-09-11 19:22:41
6,Field&Fork,C005217,2021-08-12 21:28:54,2022-07-29 21:12:00.354667
7,Field&Fork,C002681,2019-12-03 21:10:36,2020-11-07 17:59:16
8,Salinas,C002759,2019-12-05 19:33:51,2021-01-14 18:54:47
9,Salinas,C005337,2021-01-14 18:55:11,2022-07-29 21:12:00.354671


In [19]:
info_df = mark_info.merge(ref_info, on='site', how='inner')
info_df['start'] = info_df[['mark_start', 'ref_start']].max(axis=1)
info_df['end'] = info_df[['mark_end', 'ref_end']].min(axis=1)
info_df = info_df[info_df['start'] < info_df['end']].reset_index(drop=True)
info_df

,site,mark2,mark_start,mark_end,ref_end,ref_start,start,end
0,Ardoyne,C007880,2021-07-01 14:16:21,2022-07-29 21:12:00.354663,2022-07-24 23:55:00,2019-11-19 15:50:00,2021-07-01 14:16:21,2022-07-24 23:55:00
1,Ardoyne,C004146,2020-04-27 20:45:53,2020-08-27 20:10:42,2022-07-24 23:55:00,2019-11-19 15:50:00,2020-04-27 20:45:53,2020-08-27 20:10:42
2,Ardoyne,C004301,2020-08-12 19:38:53,2021-07-01 00:00:00,2022-07-24 23:55:00,2019-11-19 15:50:00,2020-08-12 19:38:53,2021-07-01 00:00:00
3,Cabure,C006056,2021-04-28 20:43:01,2022-07-29 21:12:00.354664,2021-07-03 00:00:00,2021-04-28 00:00:00,2021-04-28 20:43:01,2021-07-03 00:00:00
4,Field&Fork,C002617,2019-09-11 19:55:10,2020-04-01 01:34:53,2022-07-24 23:55:00,2019-12-16 14:45:00,2019-12-16 14:45:00,2020-04-01 01:34:53
5,Field&Fork,C005217,2021-08-12 21:28:54,2022-07-29 21:12:00.354667,2022-07-24 23:55:00,2019-12-16 14:45:00,2021-08-12 21:28:54,2022-07-24 23:55:00
6,Field&Fork,C002681,2019-12-03 21:10:36,2020-11-07 17:59:16,2022-07-24 23:55:00,2019-12-16 14:45:00,2019-12-16 14:45:00,2020-11-07 17:59:16
7,Salinas,C002759,2019-12-05 19:33:51,2021-01-14 18:54:47,2022-07-24 23:55:00,2019-12-05 00:30:00,2019-12-05 19:33:51,2021-01-14 18:54:47
8,Salinas,C005337,2021-01-14 18:55:11,2022-07-29 21:12:00.354671,2022-07-24 23:55:00,2019-12-05 00:30:00,2021-01-14 18:55:11,2022-07-24 23:55:00
9,Santa_Rosa_JimE,C002496,2019-09-04 19:29:12,2020-03-01 22:55:00,2022-07-24 23:55:00,2019-09-05 20:30:00,2019-09-05 20:30:00,2020-03-01 22:55:00


In [20]:
ref_info_condense = info_df.groupby('site').agg({'start':'min', 'end':'max'}).reset_index()
ref_info_condense

,site,start,end
0,Ardoyne,2020-04-27 20:45:53,2022-07-24 23:55:00
1,Cabure,2021-04-28 20:43:01,2021-07-03 00:00:00
2,Field&Fork,2019-12-16 14:45:00,2022-07-24 23:55:00
3,North_Canyon,2021-02-26 08:00:00,2021-04-29 06:45:00
4,Salinas,2019-12-05 19:33:51,2022-07-24 23:55:00
5,Santa_Rosa_JimE,2019-09-05 20:30:00,2022-07-24 23:55:00
6,US-Los,2020-01-06 17:09:00,2022-07-24 00:00:00
7,Yara,2021-05-13 18:15:00,2022-07-24 23:55:00
8,rainpad_portland,2022-02-25 08:00:00,2022-07-26 06:55:00


### Get mark data
1. Grab 5 min and hourly data

### Get Ref data for the selected device and time

In [21]:
def listfiles(bucket, prefix=''):
    session = Session()
    client = session.client('s3')
    """List files in specific S3 URL"""
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Prefix=prefix, Delimiter='/'):
        for content in result.get('Contents', []):
            yield content.get('Key')
#             print(f'contents are {content.get('Key')}') ##TODO
            print(content.get('Key'))
            
def get_gsheet(sheet_id, sheet_name, sheet_range):
    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    sheet_range = sheet_name + sheet_range
    creds = None
    with open('/home/ec2-user/SageMaker/adse/Met_Facts/Radiometry_Mark2/swdw cosine/token_3.pickle', 'rb') as token:
        creds = pickle.load(token)
    if not creds:
        print('token file not found')
    service = build('sheets', 'v4', credentials=creds)
    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=sheet_id,
                                range=sheet_range).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        df = pd.DataFrame(values)
        df = df.rename(columns=df.iloc[0])
        df = df.drop([0])
        df.reset_index(inplace=True, drop=True)
    return df

In [22]:
# !pip install xeus-python

In [23]:
ref_path = 's3://calval-reference-data/csv_p2'

In [24]:
def aws_csv(prefix, start_date='', end_date=''):
    # 's3://arable-adse-dev/ald/air_temp_sites/cal_sites_2019/Cabure/csv_p2/leaf wetness sensors/Cabure_20200105_20200328_p2.csv'
    # Ex: aws_csv('s3://calval-reference-data/csv_p2/Yara/growers_standard/', '2018-10-01', '2019-10-01')
    if (start_date != '') & (end_date != ''):
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
    df = pd.DataFrame()
    bucket = prefix.split('//')[1].split('/')[0]
    ignore = prefix.split('/')
    ignore = f'{ignore[0]}/{ignore[1]}/{ignore[2]}/'
    #print(bucket, f"{prefix.replace(ignore, '')}/", ignore)
    file_list = listfiles(bucket, f"{prefix.replace(ignore, '')}/")
    print(file_list)
    if 'Cabure' in prefix:
        file_list = listfiles('arable-adse-dev', 'ald/air_temp_sites/cal_sites_2019/Cabure/csv_p2/leaf wetness sensors/')
        print(file_list)
    for f in file_list:
        #print(f)
        if f.endswith(".csv"):
            if (start_date != '') & (end_date != ''):
                if 'Cabure' not in prefix:
                    f_start = pd.to_datetime((f.split('/')[-1].split('.')[0].split('_')[-2]), format='%Y%m%d%H%M')
                    f_end = pd.to_datetime((f.split('/')[-1].split('.')[0].split('_')[-1]), format='%Y%m%d%H%M')
                else:
                    f_start = pd.to_datetime((f.split('/')[-1].split('_')[-3]), format='%Y%m%d')
                    f_end = pd.to_datetime((f.split('/')[-1].split('_')[-2]), format='%Y%m%d')
                if (f_end < start_date) | (f_start > end_date):
                    continue
            if 'Cabure' in prefix:
                data_location = f's3://arable-adse-dev/{f}'
            else:
                data_location = f"{ref_path.replace('/csv_p2', '')}/{f}"
            tdf = pd.read_csv(data_location)
            df = df.append(tdf, ignore_index=True, sort=False)
    return df

def get_ref(site, start_date, end_date):
    source = sites[sites['site'] == site].source.values[0]
    prefix = f'{ref_path}/{site}/{source}'
    ref = aws_csv(prefix, start_date, end_date)
    print(prefix, ref.shape)
    return ref

In [25]:
ref_info

,ref_end,ref_start,site
0,2022-07-24 23:55:00,2019-11-19 15:50:00,Ardoyne
1,2022-07-24 23:55:00,2019-09-05 20:30:00,Santa_Rosa_JimE
2,2022-07-24 23:55:00,2019-12-16 14:45:00,Field&Fork
3,2022-07-24 23:55:00,2019-12-05 00:30:00,Salinas
4,1000-01-01 00:00:00,3000-01-01 00:00:00,UCBG
5,2022-07-24 00:00:00,2020-01-06 17:09:00,US-Los
6,2022-07-08 23:50:00,2021-01-08 00:00:00,Santa_Rosa_JimE
7,2021-07-03 00:00:00,2021-04-28 00:00:00,Cabure
8,2022-07-24 23:55:00,2021-05-13 18:15:00,Yara
9,2022-07-24 23:55:00,2021-04-22 00:00:00,UCD_Almond


# For Rajen

In [ ]:
df_final=pd.DataFrame()
count=0
for index, row in ref_info.iterrows():

    if count<=8 and row['site']!='UCBG':
        print(row['site'],row['ref_start'],row['ref_end'])
        df_test=get_ref(row['site'],row['ref_start'], row['ref_end'])
        df_final=pd.concat([df_final,df_test])
    count+=1

Ardoyne 2019-11-19 15:50:00 2022-07-24 23:55:00
<generator object listfiles at 0x7f4d085d42d0>
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_201911191550_201912312355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_201911191550_202108062355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202108070000_202108142355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202108100000_202108172355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202108160000_202111122355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202111130000_202111192355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202111200000_202111262355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202111270000_202112032355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202112040000_202112102355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202112110000_202112172355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202112180000_202112242355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202112250000_202112312355.csv
csv_p2/Ardoyne/zentra/Ardoyne__p2_5min_202201010000_202201010600.csv
csv_p2/A

In [109]:
df_test=get_ref('Santa_Rosa_JimE', '2019-09-05 20:30:00', '2022-07-24 23:55:00')

bucket is calval-reference-data, ignore ['s3:', '', 'calval-reference-data', 'csv_p2Santa_Rosa_JimE', 'zentra']
bucket is calval-reference-data zz is zentra
s3://calval-reference-data/csv_p2Santa_Rosa_JimE/zentra/
(0, 0) s3://calval-reference-data/csv_p2Santa_Rosa_JimE/zentra


In [53]:
df_test =aws_csv('s3://calval-reference-data/csv_p2/Cabure/zentra/', '2021-04-28', '2022-07-24')

bucket is calval-reference-data, ignore ['s3:', '', 'calval-reference-data', 'csv_p2', 'Cabure', 'zentra', '']
bucket is calval-reference-data zz is Cabure/zentra/
s3://calval-reference-data/csv_p2/Cabure/zentra//
visited
<generator object listfiles at 0x7f04bfeaf4d0>


In [51]:
df_test.head()

,site_id,ref_time,ref_lfw_above_canopy,ref_lfw_alternate_above_canopy,ref_lfw_alternate_in_canopy,ref_lfw_in_canopy,ref_lfw_raw_above_canopy,ref_lfw_raw_in_canopy,ref_logger_battery_percent,ref_logger_battery_voltage,ref_logger_temp,ref_p
0,Cabure,2021-04-28 21:05:00,0.0,0.0,0.0,0.0,441,440,89,7.710,19.70,99.85
1,Cabure,2021-04-28 21:10:00,0.0,0.0,0.0,0.0,441,441,88,7.707,19.36,99.85
2,Cabure,2021-04-28 21:15:00,0.0,0.0,0.0,0.0,441,441,89,7.708,19.09,99.86
3,Cabure,2021-04-28 21:20:00,0.0,0.0,0.0,0.0,441,441,88,7.706,18.83,99.86
4,Cabure,2021-04-28 21:25:00,0.0,0.0,0.0,0.0,441,441,89,7.708,18.56,99.86


In [26]:
!s3cmd ls -r s3://calval-reference-data/csv_p2/ | awk '{print $4}' > objects_in_bucket

/bin/bash: s3cmd: command not found


In [71]:
df1=aws_csv('s3://calval-reference-data/csv_p2/Yara/growers_standard/', '2018-10-01', '2019-10-01')

bucket is calval-reference-data, ignore ['s3:', '', 'calval-reference-data', 'csv_p2', 'Yara', 'growers_standard', '']
bucket is calval-reference-data zz is Yara/growers_standard/
s3://calval-reference-data/csv_p2/Yara/growers_standard//


In [20]:
# tdf = mark.copy()
# tdf['lfw'] = get_lfw_current(tdf, 'lw_temp_3')
# tdf.head()

#### Read Mark and ref and computed lfw combined df from S3 buckets from Han`s work

In [21]:
# all_in_one_df = []
# for device in site_device:
#     df = pd.read_csv(f"s3://arable-adse-dev/ald/Desktop/Met_Facts/lfw/methods_comparison_feb2022/joined_mark_and_ref/{device}.csv")
#     all_in_one_df.append(df)
# all_in_one_df = pd.concat(all_in_one_df)
# all_in_one_df.to_csv(f"s3://arable-adse-dev/ald/Desktop/Met_Facts/lfw/methods_comparison_feb2022/joined_mark_and_ref/all_in_one_df.csv", index=False) 

In [22]:
tdf = pd.read_csv(f"s3://arable-adse-dev/ald/Desktop/Met_Facts/lfw/methods_comparison_feb2022/joined_mark_and_ref/all_in_one_df.csv") 

In [23]:
tdf.columns

Index(['Unnamed: 0', 'time', 'device', 'location', 'b1dw', 'b1uw', 'b2dw',
       'b2uw', 'b3dw', 'b3uw', 'b4dw', 'b4uw', 'b5dw', 'b5uw', 'b6dw', 'b6uw',
       'b7dw', 'b7uw', 'lwdw', 'lwuw', 'lfw', 'p', 'pardw', 'paruw', 'rh',
       'swdw', 'swuw', 'tabove', 'tair', 'tbelow', 'tdew', 'dqs', 'lat',
       'long', 'create_time', 'tair_version', 'tbelow_version',
       'radcal_version', 'update_time', 'lw_temp_3', 'hour', 'precip',
       'rh_hourly', 'tbelow_corr', 'meta_lfw', 'lfw_current_algo_lw_temp_3',
       'lfw_current_algo_ml_tbelow', 'lfw_rh_threshold', 'lfw_rh_ext',
       'ref_time', 'ref_swdw', 'ref_precip', 'ref_wind_dir', 'ref_wind_speed',
       'ref_tair', 'ref_vp', 'ref_p', 'ref_orient_x', 'ref_orient_y',
       'ref_prate_max', 'ref_tsensor', 'ref_vpd', 'ref_battery_percent',
       'ref_battery_voltage', 'ref_p_abs', 'ref_logger_temp', 'ref_lfw',
       'ref_lfw_alternate', 'ref_lfw_raw', 'ref_lfw_alternate2',
       'ref_lfw_alternate3', 'ref_lfw_raw_alternate', '

In [24]:
tdf.head(5)

,Unnamed: 0,time,device,location,b1dw,b1uw,b2dw,b2uw,b3dw,b3uw,...,prev_rh1,prev_rh2,ref_swuw,ref_pardw,ref_rh,ref_tsensor_alternate,ref_soil_mp_56cm,ref_tsoil_56cm,ref_tbelow,ref_irrigation_time
0,2,2020-04-27 21:11:08,C004146,5ea6f8f762a830a90773d4dd,0.549508,0.028056,0.763365,0.054506,0.582031,0.051045,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2020-04-27 21:16:24,C004146,5ea6f8f762a830a90773d4dd,0.537033,0.027795,0.754154,0.053930,0.574793,0.050711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2020-04-27 21:21:40,C004146,5ea6f8f762a830a90773d4dd,0.542759,0.028012,0.775301,0.055370,0.589093,0.052099,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,2020-04-27 21:26:56,C004146,5ea6f8f762a830a90773d4dd,0.508811,0.027188,0.729634,0.052666,0.552550,0.049709,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,2020-04-27 21:32:12,C004146,5ea6f8f762a830a90773d4dd,0.517605,0.027600,0.751299,0.053955,0.566320,0.050816,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Method 1: RH threshold method 

In [25]:

def get_lfw_rh_threshold(df):
    rh_threshold = 0.87
    return df.rh > rh_threshold

In [26]:
tdf['lfw']= get_lfw_rh_threshold(tdf)

In [27]:
tdf[['device', 'time', 'lfw']]

,device,time,lfw
0,C004146,2020-04-27 21:11:08,False
1,C004146,2020-04-27 21:16:24,False
2,C004146,2020-04-27 21:21:40,False
3,C004146,2020-04-27 21:26:56,False
4,C004146,2020-04-27 21:32:12,False
...,...,...,...
1236081,C006740,2022-01-28 23:33:48,False
1236082,C006740,2022-01-28 23:38:48,False
1236083,C006740,2022-01-28 23:43:50,False
1236084,C006740,2022-01-28 23:48:50,False


#### Method 2: RH extented Method

In [28]:
def get_lfw_rh_ext(df):
    """The input df should have column time that is sorted so that it can be used
    in function time diff later
    """
    high_ext_rh = 0.87
    low_ext_rh = 0.70
    increase_rate = 0.03
    decrease_rate = -0.02
    df['meta_lfw'] = df.rh > high_ext_rh
    df.loc[df.rh < low_ext_rh, 'meta_lfw'] = False
    
    # rate
    
    df = df.set_index(pd.DatetimeIndex(df['time']))
    df['rh_pct_change'] = df['rh'].rolling('10min').apply(my_pct_change)
    df.loc[(df.rh_pct_change > increase_rate) & (df.rh.between(low_ext_rh, high_ext_rh)), 'meta_lfw'] = True
    df.loc[(df.rh_pct_change < decrease_rate) & (df.rh.between(low_ext_rh, high_ext_rh)), 'meta_lfw'] = False
    df = df.resample('5min').first()
    df['lfw_rh_ext'] = df.meta_lfw.fillna(method='ffill', limit=12)
    df = df.reset_index(drop=True)
    return df[['time', 'lfw_rh_ext']]

#### Method 3 & 4: Current method used in production with two different tbelow ( raw tbelow and ML tbelow)

In [29]:
# 3 and 4
def get_lfw_current(df, which_tbelow):
    """Takes in df that have calibrated data, lw_temp3 from raw and outer joined with hourly precip
    Can be used to calculate lfw in 1 and 2, which 1 using tbelow instead of lw_temp_3
    """
    tdew_offset = 0.99
    low_rh_threshold = 0.90
    high_rh_threshold = 0.93
    
    if which_tbelow == 'lw_temp_3':
        # make tbelow_V1
        df['tbelow_corr'] = make_tbelow_v1(df.lw_temp_3)
        # make alter dpd with default offsets
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow_corr', tdew_offset, low_rh_threshold)
    elif which_tbelow == 'tbelow':
        # make alter dpd
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow', tdew_offset, low_rh_threshold)
    else:
        raise ValueError(f'not expecting tbelow type to be {which_tbelow}') 
    
    # cooperate ^ with hourly data to make final lfw
    return (df.precip > 0) | (df.rh_hourly > high_rh_threshold) | (df.meta_lfw > 0)

def make_tbelow_v1(sr):
    """Takes lw_temp_3 and returns Tbelow_corr"""
    za = np.array([0.01935698, 0.8618187, -2.34398788])
    return za[0] * (sr ** 2) + za[1] * sr + za[2]

def alter_dew_point_depression(df, tbelow_type, tdew_offset, low_rh_threshold):
    return (df[tbelow_type] <= df.tdew + tdew_offset) \
                    & (df.rh > low_rh_threshold)

#### call function with raw tbelow

In [24]:
tdf = mark.copy()
tdf['lfw'] = get_lfw_current(tdf, 'lw_temp_3')
tdf.head()

NameError: name 'mark' is not defined

### Call function with ML tbelow

In [23]:
df = mark.copy()
df['lfw'] = get_lfw_current(df, 'tbelow')
df.head()

NameError: name 'mark' is not defined